In [131]:
import pandas as pd
import numpy as np

In [132]:
f = open('KJV.txt','r')
bible_text = f.read()
bible_text[:1000]

"\ufeffThe Project Gutenberg EBook of The King James Bible\n\n\n**********************************************************************\nEBOOK (#10) WAS ONE OF PROJECT GUTENBERG'S EARLY FILES PRODUCED AT A\nTIME WHEN PROOFING METHODS AND TOOLS WERE NOT WELL DEVELOPED. THERE IS\nAN IMPROVED EDITION OF THIS TITLE WHICH MAY VIEWED AT EBOOK http://www.gutenberg.org/files/10900/10900-h/10900-h.htm\n(There is no text file for this ebook)\n**********************************************************************\n\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\nTitle: The King James Bible\n\nRelease Date: March 2, 2011 [EBook #10]\n[This King James Bible was orginally posted by Project Gutenberg\nin late 1989]\n\nLanguage: English\n\n\n*** START OF THIS PROJECT GUTENBERG EBOOK THE KING JA

In [133]:
bible_books = ['Genesis', 'Exodus', 'Leviticus','Numbers','Deuteronomy', 'Joshua',
                 'Judges', 'Ruth', 'Samuel', '2 Samuel', '1 Kings', '2 Kings',
                 '1 Chronicles', '2 Chronicles', 'Ezra', 'Nehemiah', 'Esther', 'Job',
                  'Psalms','Proverbs', 'Ecclesiastes', 'Song of Solomon', 'Isaiah', 'Jeremiah',
                  'Lamentations','Ezekiel', 'Daniel', 'Hosea', 'Joel', 'Amos',
                  'Obadiah', 'Jonah', 'Micah', 'Nahum', 'Habbakkuk', 'Zephaniah',
                  'Haggai', 'Zechariah', 'Malachi', 'Matthew', 'Mark', 'Luke',
                  'John','Acs', 'Romans', '1 Corinthians', '2 Corinthians', 'Galatians',
                  'Ephesians','Philippians', 'Colossians', '1 Thessalonians', '2 Thessalonians', '1 Timothy',
                  '2 Timothy','Titus', 'Philemon', 'Hebrews', 'James', '1 Peter',
                  '2 Peter','1 John', '2 John', '3 John', 'Jude', 'Revelation']

In [134]:
book_in_bible = [book in bible_text for book in bible_books]
pd.concat([pd.Series(bible_books), pd.Series(book_in_bible)], axis =1).head(20)

,0,1
0,Genesis,True
1,Exodus,True
2,Leviticus,True
3,Numbers,True
4,Deuteronomy,True
5,Joshua,True
6,Judges,True
7,Ruth,True
8,Samuel,True
9,2 Samuel,False


In [135]:
positions = [bible_text.find(book) for book in bible_books]
positions

[1090,
 207739,
 384862,
 517103,
 700707,
 277963,
 774591,
 1059638,
 1073136,
 -1,
 2309361,
 2170690,
 -1,
 -1,
 1342710,
 1839010,
 1938151,
 33793,
 2070262,
 2311144,
 2396887,
 2426531,
 1541863,
 1566109,
 2875341,
 2894351,
 1585241,
 3171691,
 1099080,
 3211057,
 1418599,
 1517912,
 1034317,
 3261187,
 -1,
 1573415,
 1852281,
 1593078,
 3325504,
 3335086,
 1258725,
 3549640,
 3341287,
 -1,
 3750078,
 -1,
 -1,
 4069171,
 3886566,
 4103400,
 4115570,
 -1,
 -1,
 -1,
 -1,
 4040329,
 4171287,
 159299,
 41,
 3794334,
 -1,
 -1,
 -1,
 -1,
 1853497,
 4269040]

While the name of most of the books in the bible may be located in the text, but their occurences don't seem to always align with the start of the book. This is easy to see because the positions of each of the words are not in ascending order. I could split the text manually but I wonder if there is a way to do so more intelligently...

But let's have a look at some of the locations of the words.

In [136]:
snippets = []
for position in positions:
    if position != -1:
        snippet = bible_text[position-30:position+30]
    else:
        snippet = ''
    snippets.append(snippet)
snippets

[' First Book of Moses:  Called Genesis\n\n\n1:1 In the beginning',
 'Second Book of Moses:  Called Exodus\n\n\n1:1 Now these are the',
 ' Third Book of Moses:  Called Leviticus\n\n\n1:1 And the LORD c',
 'Fourth Book of Moses:  Called Numbers\n\n\n1:1 And the LORD spa',
 ' Fifth Book of Moses:  Called Deuteronomy\n\n\n1:1 These be the',
 'im.\n\n17:9 And Moses said unto Joshua, Choose us out men, and',
 'ch he hath given thee.\n\n16:18 Judges and officers shalt thou',
 'his own eyes.\n\n\n\n\nThe Book of Ruth\n\n\n1:1 Now it came to pass',
 ' David.\n\n\n\n\nThe First Book of Samuel\n\nOtherwise Called:\n\nThe',
 '',
 'hings, and flying fowl:\n\n148:11 Kings of the earth, and all ',
 'those that\npublished it.\n\n68:12 Kings of armies did flee apa',
 '',
 '',
 ' than all men; than Ethan the Ezrahite, and\nHeman, and Chalc',
 'came with\nZerubbabel: Jeshua, Nehemiah, Seraiah, Reelaiah, M',
 'od, for good.\n\n\n\n\nThe Book of Esther\n\n\n1:1 Now it came to pa',
 '9 And Ophir, and Havil

While locating the book name works in most cases, in some cases it ends up picking a false positive. One thing we can do is look at the text immediately preceeding the first verse of each book. The first verse of each book will have a standard format so that should be easy

In [137]:
#Extracting verses
import re

surrounding_texts = []

for match in re.finditer("\n1:1 ", bible_text):
    surrounding_text = bible_text[match.start()-90:match.end()]
    surrounding_texts.append(surrounding_text)

surrounding_texts



['ment of the King James Version of the Bible\n\n\n\n\nThe First Book of Moses:  Called Genesis\n\n\n1:1 ',
 'd him, and he was put in a coffin in Egypt.\n\n\n\n\nThe Second Book of Moses:  Called Exodus\n\n\n1:1 ',
 'of Israel,\nthroughout all their journeys.\n\n\n\n\nThe Third Book of Moses:  Called Leviticus\n\n\n1:1 ',
 'for\nthe children of Israel in mount Sinai.\n\n\n\n\nThe Fourth Book of Moses:  Called Numbers\n\n\n1:1 ',
 '\nplains of Moab by Jordan near Jericho.\n\n\n\n\nThe Fifth Book of Moses:  Called Deuteronomy\n\n\n1:1 ',
 'l the great terror which Moses shewed\nin the sight of all Israel.\n\n\n\n\nThe Book of Joshua\n\n\n1:1 ',
 'tained to Phinehas his son, which was given him in mount\nEphraim.\n\n\n\n\nThe Book of Judges\n\n\n1:1 ',
 'king in Israel: every man did that\nwhich was right in his own eyes.\n\n\n\n\nThe Book of Ruth\n\n\n1:1 ',
 'egat David.\n\n\n\n\nThe First Book of Samuel\n\nOtherwise Called:\n\nThe First Book of the Kings\n\n\n1:1 ',
 'ven days.\n\n\n\

The title of each book is preceeded by five newline characters ('\n') and is followed by three newline characters. We can use positive look arounds to extract the text in the middle. 

In [138]:
pattern = '(?<=\n\n\n\n\n)(.+)(?=\n\n\n)'
book_names = re.findall(pattern,bible_text)
book_names

['The Old Testament of the King James Version of the Bible',
 'The First Book of Moses:  Called Genesis',
 'The Second Book of Moses:  Called Exodus',
 'The Third Book of Moses:  Called Leviticus',
 'The Fourth Book of Moses:  Called Numbers',
 'The Fifth Book of Moses:  Called Deuteronomy',
 'The Book of Joshua',
 'The Book of Judges',
 'The Book of Ruth',
 'The First Book of the Chronicles',
 'The Second Book of the Chronicles',
 'Ezra',
 'The Book of Nehemiah',
 'The Book of Esther',
 'The Book of Job',
 'The Book of Psalms',
 'The Proverbs',
 'The Song of Solomon',
 'The Book of the Prophet Isaiah',
 'The Book of the Prophet Jeremiah',
 'The Lamentations of Jeremiah',
 'The Book of the Prophet Ezekiel',
 'The Book of Daniel',
 'Hosea',
 'Joel',
 'Amos',
 'Obadiah',
 'Jonah',
 'Micah',
 'Nahum',
 'Habakkuk',
 'Zephaniah',
 'Haggai',
 'Zechariah',
 'Malachi',
 'The New Testament of the King James Bible',
 'The Gospel According to Saint Matthew',
 'The Gospel According to Saint Mark',

Ok, looks mostly good so far. Let's check the length.

In [139]:
len(book_names)

63

Uh oh! Looks like we're missing some. Upon closer inspection we're missing the books of 1,2 Samuel, 1,2 Kings and Ecclesiastes. These are formatted slightly differently with new line characters in the title as well. We can use a more complicated regex to extract these books as well

In [140]:
#pattern = '(?<=\n\n\n\n\n)(.+?\n?\n?\n?.*?\n?\n?.*)(?=\n\n\n)'
pattern = '(?<=\n\n\n\n\n)(.+?\n?\n?.*?\n?\n?.*)(?=\n\n\n)'
book_names = re.findall(pattern,bible_text)
len(book_names)

68

There are 66 books in the bible with two additional entries for the start of the Old and New Testament. We'll delete them from the list and our list of bible books should be complete.

In [141]:
del book_names [0], book_names[38]
book_names

['The First Book of Moses:  Called Genesis',
 'The Second Book of Moses:  Called Exodus',
 'The Third Book of Moses:  Called Leviticus',
 'The Fourth Book of Moses:  Called Numbers',
 'The Fifth Book of Moses:  Called Deuteronomy',
 'The Book of Joshua',
 'The Book of Judges',
 'The Book of Ruth',
 'The First Book of Samuel\n\nOtherwise Called:\n\nThe First Book of the Kings',
 'The Second Book of Samuel\n\nOtherwise Called:\n\nThe Second Book of the Kings',
 'The First Book of the Kings\n\nCommonly Called:\n\nThe Third Book of the Kings',
 'The Second Book of the Kings\n\nCommonly Called:\n\nThe Fourth Book of the Kings',
 'The First Book of the Chronicles',
 'The Second Book of the Chronicles',
 'Ezra',
 'The Book of Nehemiah',
 'The Book of Esther',
 'The Book of Job',
 'The Book of Psalms',
 'The Proverbs',
 'Ecclesiastes\n\nor\n\nThe Preacher',
 'The Song of Solomon',
 'The Book of the Prophet Isaiah',
 'The Book of the Prophet Jeremiah',
 'The Lamentations of Jeremiah',
 'The Boo

In [142]:
len(book_names)

66

Let's find the position of these book names so we can segment the rest of the bible text into the relevant books

In [143]:
texts = []
for book in book_names:
    text = re.split(book, bible_text)
    texts.append(text[1])

The next stage we need to move onto is to 